In [31]:
import torch
import os
import gc
import torch.nn as nn
import numpy as np
from torch.utils.data import Dataset, DataLoader
from torch import optim
#os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"

seed = 0
np.random.seed(seed)
torch.manual_seed(seed)

Data_train = np.load('timit_11/train_11.npy')
Data_train_label = np.load('timit_11/train_label_11.npy')
Data_test = np.load('timit_11/test_11.npy')

class TIMITData(Dataset) :
    def __init__(self, X, y = None) :
        self.data = torch.from_numpy(X).float()
        if y is not None :
            y = y.astype(np.int32)
            self.label = torch.LongTensor(y)
        else:
            self.label = None
    def __getitem__(self, item) :
        if self.label is not None :
            return self.data[item], self.label[item]
        else:
            return self.data[item]
    def __len__(self) :
        return len(self.data)
VAL_RATE = 0.2
percent = int(Data_train.shape[0] * (1 - VAL_RATE))
train_x, train_y, val_x, val_y = Data_train[ : percent], Data_train_label[ : percent], Data_train[percent :], Data_train_label[percent :]

Batch_size = 270
train_data = TIMITData(train_x, train_y)
val_data = TIMITData(val_x, val_y)
train_loader = DataLoader(train_data, batch_size=Batch_size, shuffle=True)
val_loader = DataLoader(val_data, batch_size=Batch_size, shuffle=False)

del Data_train, Data_train_label, train_x, train_y, val_x, val_y
gc.collect()

class Classifier(nn.Module) :
    def __init__(self) :
        super(Classifier, self).__init__()
        self.net = nn.Sequential(nn.Linear(429, 1024),
                                nn.Sigmoid(),
                                nn.Linear(1024, 512),
                                nn.Sigmoid(),
                                nn.Linear(512, 128),
                                nn.Sigmoid(),
                                nn.Linear(128, 39))
    def forward(self, x) :
        return self.net(x)
def get_device() :
    return 'cuda'if torch.cuda.is_available() else "cpu"

device = get_device()
Learning_rate = 0.001
max_epoch = 20
model = Classifier().to(device)
critertion = nn.CrossEntropyLoss()
optimization = optim.Adam(model.parameters(), lr=Learning_rate)
model_path = './model.ckpt'
best_acc = 0.0

for epoch in range(max_epoch) :
    train_loss = 0.0
    train_acc = 0.0
    val_loss = 0.0
    val_acc = 0.0
    model.train()
    for x, label in train_loader:
        x = x.to(device)
        label = label.to(device)
        optimization.zero_grad()
        output = model(x)
        #print(output.shape)
        #print(label.shape)
        loss = critertion(output, label)
        _, train_pred = torch.max(output, 1)
        loss.backward()
        optimization.step()
        
        train_loss += loss.item()
        num_correct = (train_pred.cpu() == label.cpu()).sum().item()
        train_acc += num_correct / x.shape[0]
        
    if len(val_data) > 0:
        model.eval()
        with torch.no_grad() :
            for x, label in val_loader :
                x = x.to(device) 
                label = label.to(device)
                output = model(x)
                loss = critertion(output, label)
                _, val_pred = torch.max(output, 1)
                
                val_loss += loss.item()
                num_correct = (val_pred.cpu() == label.cpu()).sum().item()
                val_acc += num_correct / x.shape[0]
            print('[{:03d}/{:03d}] Train Acc: {:3.6f} Loss: {:3.6f} | Val Acc: {:3.6f} loss: {:3.6f}'.format(epoch + 1, max_epoch, 
                                                                                                                train_acc / len(train_loader), train_loss / len(train_loader), 
                                                                                                                 val_acc / len(val_loader), val_loss / len(val_loader)))
            if val_acc > best_acc :
                best_acc = val_acc
                torch.save(model.state_dict(), model_path)
    else:
        print('[{:03d}/{:03d}] Train Acc: {:3.6f} Loss: {:3.6f}'.format(
            epoch + 1, num_epoch, train_acc/len(train_set), train_loss/len(train_loader)
        ))
if len(val_data) == 0 :
      torch.save(model.state_dict(), model_path)

test_data = TIMITData(Data_test, y = None)
test_loader = DataLoader(test_data, batch_size = Batch_size, shuffle=False)
predict = []
model = Classifier().to(device)
model.load_state_dict(torch.load(model_path))
model.eval()

with torch.no_grad() :
    for x in test_loader :
        x = x.to(device)
        output = model(x)
        _, test_pred = torch.max(output, 1)
    
        for i in test_pred.cpu().numpy() :
            predict.append(i)
with open('prediction.csv', 'w') as file :
    file.write('Id,Class\n')
    for i, Class in enumerate(predict) :
        file.write('{},{}\n'.format(i, Class))

[001/003] Train Acc: 0.575048 Loss: 1.403044 | Val Acc: 0.665181 loss: 1.069692
[002/003] Train Acc: 0.686195 Loss: 0.985308 | Val Acc: 0.688143 loss: 0.975280
[003/003] Train Acc: 0.715884 Loss: 0.878557 | Val Acc: 0.697087 loss: 0.943670
